# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
import lightgbm as lgb


clf = lgb.LGBMClassifier(objective='multiclass',
                         num_leaves=40,
                         learning_rate=0.05,
                         n_estimators=128)


clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LGBMClassifier(learning_rate=0.05, n_estimators=128, num_leaves=40,
               objective='multiclass')

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	namespace LGBM_Tree_0_0 {
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 3 ,  {-1.1361122895063 }} ,
				{ 4 ,  {-1.0292372871174578 }} ,
				{ 5 ,  {-1.0236122869917292 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
			int lNodeIndex = ( ( Feature_2 <= 3.1500000000000004 ) ? ( ( ( Feature_1 <= 3.35 ) ? ( 4 ) : ( 5 ) ) ) : ( 3 ) );
		
			return lNodeIndex;
		}
		
	
		std::vector<std::string> get_input_names(){
			std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };
	
			return lFeatures;
		}
	
		std::vector<std::any> get_classes(){
			std::vector<std::any> lClasses = { 0, 1, 2 };
	
			return lClasses;
		}
	
		std::vector<std::string> get_output_names(){
			std::vector<std::string> lOutputs = { 
				"Score_0", "Score_1", "Score_2",
				"Proba_0", "Proba_1", "Proba_2",
				"LogProba_0", "LogProb

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140292439891488.exe /tmp/sklearn2sql_cpp_140292439891488.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,0.99814700295762,0.00104791543424,0.00080508160814,-0.00185471596517,-6.86095238885582,-7.12456690911624,0,0.99814700295762
1,,,,0.99617192863396,0.00154289590080,0.00228517546524,-0.00383541718411,-6.47409417333403,-6.08131246754264,0,0.99617192863396
2,,,,0.99689394801660,0.00154401418241,0.00156203780099,-0.00311088577479,-6.47336964192472,-6.46176402747989,0,0.99689394801660
3,,,,0.99649449861944,0.00141991248199,0.00208558889857,-0.00351166004755,-6.55716004166921,-6.17270401825456,0,0.99649449861944
4,,,,0.99855684185019,0.00080827065106,0.00063488749875,-0.00144420050551,-7.12061359135046,-7.36206274208584,0,0.99855684185019
5,,,,0.99851992348255,0.00096443466153,0.00051564185592,-0.00148117291267,-6.94396847127506,-7.5700981111

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.002230,0.011498,0.986272,-6.105914,-4.465550,-0.013823,2,0.986272
74,74,NaN,NaN,NaN,0.001832,0.993867,0.004301,-6.302109,-0.006152,-5.448895,1,0.993867
9,9,NaN,NaN,NaN,0.996494,0.001420,0.002086,-0.003512,-6.557160,-6.172704,0,0.996494
88,88,NaN,NaN,NaN,0.001880,0.995098,0.003022,-6.276660,-0.004914,-5.801753,1,0.995098
25,25,NaN,NaN,NaN,0.995954,0.001891,0.002155,-0.004054,-6.270649,-6.139953,0,0.995954
5,5,NaN,NaN,NaN,0.998520,0.000964,0.000516,-0.001481,-6.943968,-7.570098,0,0.998520
48,48,NaN,NaN,NaN,0.998327,0.000964,0.000708,-0.001674,-6.944161,-7.252395,0,0.998327
117,117,NaN,NaN,NaN,0.001490,0.004225,0.994285,-6.508702,-5.466810,-0.005731,2,0.994285
83,83,NaN,NaN,NaN,0.008408,0.438938,0.552653,-4.778559,-0.823396,-0.593024,2,0.552653
105,105,NaN,NaN,NaN,0.000936,0.004227,0.994837,-6.974255,-5.466255,-0.005176,2,0.994837


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.002230,0.011498,0.986272,NaN,NaN,NaN,2
74,74,NaN,NaN,NaN,0.001832,0.993867,0.004301,NaN,NaN,NaN,1
9,9,NaN,NaN,NaN,0.996494,0.001420,0.002086,NaN,NaN,NaN,0
88,88,NaN,NaN,NaN,0.001880,0.995098,0.003022,NaN,NaN,NaN,1
25,25,NaN,NaN,NaN,0.995954,0.001891,0.002155,NaN,NaN,NaN,0
5,5,NaN,NaN,NaN,0.998520,0.000964,0.000516,NaN,NaN,NaN,0
48,48,NaN,NaN,NaN,0.998327,0.000964,0.000708,NaN,NaN,NaN,0
117,117,NaN,NaN,NaN,0.001490,0.004225,0.994285,NaN,NaN,NaN,2
83,83,NaN,NaN,NaN,0.008408,0.438938,0.552653,NaN,NaN,NaN,2
105,105,NaN,NaN,NaN,0.000936,0.004227,0.994837,NaN,NaN,NaN,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,0.002230,0.011498,0.986272,NaN,NaN,NaN,...,NaN,NaN,0.002230,0.011498,0.986272,-6.105914,-4.465550,-0.013823,2,0.986272
74,74,NaN,NaN,NaN,0.001832,0.993867,0.004301,NaN,NaN,NaN,...,NaN,NaN,0.001832,0.993867,0.004301,-6.302109,-0.006152,-5.448895,1,0.993867
9,9,NaN,NaN,NaN,0.996494,0.001420,0.002086,NaN,NaN,NaN,...,NaN,NaN,0.996494,0.001420,0.002086,-0.003512,-6.557160,-6.172704,0,0.996494
88,88,NaN,NaN,NaN,0.001880,0.995098,0.003022,NaN,NaN,NaN,...,NaN,NaN,0.001880,0.995098,0.003022,-6.276660,-0.004914,-5.801753,1,0.995098
25,25,NaN,NaN,NaN,0.995954,0.001891,0.002155,NaN,NaN,NaN,...,NaN,NaN,0.995954,0.001891,0.002155,-0.004054,-6.270649,-6.139953,0,0.995954
5,5,NaN,NaN,NaN,0.998520,0.000964,0.000516,NaN,NaN,NaN,...,NaN,NaN,0.998520,0.000964,0.000516,-0.001481,-6.943968,-7.570098,0,0.998520
48,48,NaN,NaN,NaN,0.998327,0.000964,0.000708,NaN,NaN,NaN,...,NaN,NaN,0.998327,0.000964,0.000708,-0.001674,-6.944161,-7.252395,0,0.998327
117,117,NaN,NaN,NaN,0.001490,0.004225,0.994285,NaN,NaN,NaN,...,NaN,NaN,0.001490,0.004225,0.994285,-6.508702,-5.466810,-0.005731,2,0.994285
83,83,NaN,NaN,NaN,0.008408,0.438938,0.552653,NaN,NaN,NaN,...,NaN,NaN,0.008408,0.438938,0.552653,-4.778559,-0.823396,-0.593024,2,0.552653
105,105,NaN,NaN,NaN,0.000936,0.004227,0.994837,NaN,NaN,NaN,...,NaN,NaN,0.000936,0.004227,0.994837,-6.974255,-5.466255,-0.005176,2,0.994837


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
